# Imports/Definitions

In [ ]:
import PIL.Image
import PIL.ImageOps
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from rpyc.utils.classic import obtain
plt.rcParams["image.origin"] = 'lower'
import numpy as np
import scipy
from PIL import Image, ImageOps

#Interactive plotting
#%matplotlib qt

from scipy.optimize import curve_fit
from qudi.logic.correlation_logic import Rotation
import skimage

In [ ]:
def show(image, vmax=60000, title=None, label='\si{Photons\per\second}'):
    # get np array from rpyc netref object
    image = obtain(image)
    # y_range, x_range = image._image.shape
    tick_extent = [0, image.range_xy[0]*1e6, 0, image.range_xy[1]*1e6]
    tick_extent = np.array(tick_extent)
    #fig_size = calc_plot_size(image._image)
    fig = plt.figure()
    if vmax is None:
        vmax = np.max(image.array)
    if type(image.array) is not PIL.Image.Image and image.array.dtype == np.dtype('uint8') and vmax > 255:
        vmax = 255
    ax = plt.subplot()
    image_plot = plt.imshow(image.array, vmax=vmax, extent=tick_extent)
    plt.xlabel(r'X position (\si{\micro\metre})')
    plt.ylabel(r'Y position (\si{\micro\metre})')
    plt.title(title)

    image_plot.set_cmap('inferno')
    div = make_axes_locatable(ax)
    cax = div.append_axes('right', size='5%', pad=0.1)
    # plt.colorbar(shrink=.5)
    colorbar = plt.colorbar(image_plot, cax=cax)
    colorbar.set_label(label)
    colorbar.outline.set_edgecolor('black')
    # plt.tight_layout()
    plt.show()

In [ ]:
# white_1='red', white_2='blue'
def imshow_composite(xy_1, xy_2, trans_vec, white_1='yellow', white_2='red', black='black', plot=False, title=None,
                     ratio=None, alpha=128, lines_color=None, xlabel=None, ylabel=None, edges=True, return_composite = False):
    """shows the composite image of array xy_1 and xy_2 after translating xy_1 according to the given translation
    vector and then overlapping xy_2 over xy_1. The color of the two arrays can be individually set with white_1 and
    white_2. Arrays don't have to be of same shape. """

    # convert the arrays to Images, change the color and make im_after transparent
    im_before = Image.fromarray(xy_1).convert('L')
    im_after = Image.fromarray(xy_2).convert('L')
    im_before = ImageOps.colorize(im_before, black=black, white=white_1)
    im_after = ImageOps.colorize(im_after, black=black, white=white_2)
    im_after.putalpha(alpha)
    im_before = im_before.convert('RGBA')

    # now translate im_before according to the translation vector and paste in im_after:
    im_before_width, im_before_height = im_before.size
    im_after_width, im_after_height = im_after.size

    # if translation in one direction is positive, im_before is put at the end of im_before_trans , else at the
    # beginning (0). For im_after the opposite is true ( because im_before is translated, im_after stays stationary)
    if trans_vec[0] > 0:
        before_x_start = trans_vec[0]
        after_x_start = 0
        im_composite_width = max(im_before_width + trans_vec[0], im_after_width)
    else:
        before_x_start = 0
        after_x_start = np.abs(trans_vec[0])
        im_composite_width = max(im_before_width, np.abs(trans_vec[0]) + im_after_width)
    if trans_vec[1] > 0:
        before_y_start = trans_vec[1]
        after_y_start = 0
        im_composite_height = max(im_before_height + trans_vec[1], im_after_height)
    else:
        before_y_start = 0
        after_y_start = np.abs(trans_vec[1])
        im_composite_height = max(im_before_height, np.abs(trans_vec[1]) + im_after_height)

    # new image with bigger size because of translation
    im_composite = Image.new("RGBA",
                             (im_composite_width,
                              im_composite_height))

    im_composite.paste(im_before, (before_x_start, before_y_start))
    im_composite.alpha_composite(im_after, dest=(after_x_start, after_y_start))
    # plot the result with PIL
    im_composite.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM).show(title=title)
    if return_composite:
        return im_composite

In [ ]:
# functions for rotation detection
def gaussian(x, a, x_0, sigma, c):
    exponent = (x-x_0)**2/(2*sigma**2)
    return a*np.exp(-exponent)+c

def fit_sweep(results, plot=True):
    values = [result['sweep_value'] for result in results]
    maxima = [result['max_correlation_value'] for result in results]
    max_index = np.argmax(maxima)

    # fit gaussian
    param_est = [np.max(maxima) - np.min(maxima), values[np.argmax(maxima)], 1, np.min(maxima)]
    fit = curve_fit(gaussian, values, maxima, param_est)
    fit_params = fit[0]

    # x_0 from fit is value with maximum max correlation value
    value_fit_max = fit_params[1]
    # amplitude a + displacement c is the maximum max correlation value of the fit
    fit_max = fit_params[0] + fit_params[3]

    values_for_fit = np.linspace(np.min(values), np.max(values), 10 * len(values))
    evaluated_fit = gaussian(values_for_fit, *fit_params)
    if plot:
        plot_sweep(value_fit_max, fit_max, values_for_fit, evaluated_fit, values, maxima)
    # return rotation with maximum fit value
    return value_fit_max

def plot_sweep(value_fit_max, fit_max, values_for_fit, evaluated_fit, values, maxima):
    plt.figure()
#   plt.plot(x,y, label='Interpolation')
    plt.plot(values_for_fit, evaluated_fit, 'c--', label='Gaussian fit')
#     plt.plot(values[max_index], maxima[max_index], 'kx', label=f'Max maxima at {values[max_index]:.4f}° rotation')
    plt.vlines(value_fit_max, np.min(maxima), fit_max*1.5, linestyles='dashed', colors='grey',
                   label=f'Gaussian maximum at {value_fit_max:.2f}° rotation')
    plt.plot(values, maxima, 'r.', label='Correlation maxima for different values')
    plt.xlabel('Rotation (°\,)')
    plt.ylabel('Correlation maxima (\si{(photons\per\second)\squared})')
    plt.legend()
    plt.grid()
    plt.show()
#   print(f'Fit: {fit_params}')
#     print(f'Maximum correlation maxima {maxima[max_index]:.1f} for {values[max_index]:.3f}° rotation')
#    print(f'Maximum correlation maxima {fit_max:.1f} for {value_fit_max:.3f}° rotation with fit')

# Import correlation images

In [ ]:
# import image 1 from absolute path
path_1 = r""
correlation_logic.import_image(1, path_1)

#import image 2 from absolute path
path_2 = r""
correlation_logic.import_image(2, path_2)

# Correlate images without rotation detection

In [ ]:
correlation_logic.default_init_process_steps()
correlation_logic.run()

# results are stored in correlation_logic.result_dict:
translation_pixels = obtain(correlation_logic.result_dict['translation_vector_xy_pixels'])
translation_meters = obtain(correlation_logic.result_dict['translation_vector_xy_meters'])
print(f'Found translation: {translation_pixels} pixels or {translation_meters*10**6} micrometers')

In [ ]:
# show image composite
display_image_1 = obtain(correlation_logic.display_image_1)
display_image_2 = obtain(correlation_logic.display_image_2)
comp = imshow_composite(display_image_1.array, display_image_2.array, translation_pixels)

In [ ]:
# show image 1 or 2
show(correlation_logic.display_image_1)

In [ ]:
# plot correlation image
show(correlation_logic.result_dict['correlation_image'], label='\si{(Photons\per\second)^2}')

# Find rotation and correlate with corrected rotation

## Start by finding the rotation

In [ ]:
# print current step list and their state
print([(step.name, step.parameters['enabled']) for step in correlation_logic.process_steps])

In [ ]:
# initialize step list for rotation
correlation_logic.rotation_init_process_steps()
print([(step.name, step.parameters['enabled']) for step in correlation_logic.process_steps])

In [ ]:
# sweep through the rotation angle
rot_min = -5
rot_max = 5
number_of_rotations = 25
results = correlation_logic.sweep('Rotation','rotation_angle', np.linspace(rot_min, rot_max, number_of_rotations))

In [ ]:
# get the results for further calculations
results = obtain(results)

In [ ]:
# fit the correlation maxima for the different rotations, if no optimal parameters are found go to next code block
detected_rotation = fit_sweep(results)

In [ ]:
# if no optimal parameters were found, one can plot the values by hand and maybe find the rotation and set it manually
values = [result['sweep_value'] for result in results]
maxima = [result['max_correlation_value'] for result in results]
plt.plot(values, maxima, 'x--')
print(f'Maximum for a rotation of {values[np.argmax(maxima)]}°')
# can set detected_rotation to maximum value, but only if there is a clear peak in the plot and the fit_sweep function wasn't successfull
# detected_rotation = values[np.argmax(maxima)]

## Now correct detected rotation and correlate

In [ ]:
# initialize process steps to default, but put a Rotation step, which corrects the detected rotation, as a first step 
correlation_logic.default_init_process_steps()
rotation_step = Rotation({'index': -1, 'enabled': True, 'rotation_angle': detected_rotation})
correlation_logic.add_process_step(rotation_step)
print(f'Rotation set to {detected_rotation:.3f}°')
print([(step.name, step.parameters['enabled']) for step in correlation_logic.process_steps])

In [ ]:
# run correlation
correlation_logic.run()
print(correlation_logic.result_dict)
translation_pixels = obtain(correlation_logic.result_dict['translation_vector_xy_pixels'])
translation_meters = obtain(correlation_logic.result_dict['translation_vector_xy_meters'])
print(f'Found translation: {translation_pixels} pixels or {translation_meters*10**6} micrometers')

In [ ]:
# show image composite
display_image_1 = obtain(correlation_logic.display_image_1)
display_image_2 = obtain(correlation_logic.display_image_2)
comp = imshow_composite(display_image_1.array, display_image_2.array, translation_pixels)

In [ ]:
# plot correlation image
plt.imshow(correlation_logic.result_dict['correlation_image'].array)